In [59]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

In [2]:
import adagram
import gensim
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_rt = pd.read_csv('news_texts.csv')

In [4]:
data_rt.dropna(inplace=True)

In [6]:
X.shape

(7212, 1000)

In [89]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [90]:
data[:10]

,text_1,text_2,label
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,1
6,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,-1
7,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,0
8,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,-1
9,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,-1


In [91]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

### SVD

In [168]:
tfidf = TfidfVectorizer(max_features=7227)
X = tfidf.fit_transform(data['text_1_norm'])
Y = tfidf.fit_transform(data['text_2_norm'])

In [169]:
svd = TruncatedSVD(50)
svd_X = svd.fit(X)
svd_Y = svd.fit(Y)

In [170]:
id2vec_svd_X = svd_X.components_.T
id2vec_svd_Y = svd_Y.components_.T

In [171]:
len(id2vec_svd_X)

7227

In [172]:
cos_svd = cosine_distances(id2vec_svd_X,id2vec_svd_Y)

### NMF

In [174]:
nmf = NMF(50)
nmf_X = nmf.fit(X)
nmf_Y = nmf.fit(Y)

In [175]:
id2vec_nmf_X = nmf_X.components_.T
id2vec_nmf_Y = nmf_Y.components_.T

In [176]:
cos_nmf = cosine_distances(id2vec_nmf_X,id2vec_nmf_Y)

### Fasttext

In [107]:
fast_text = gensim.models.FastText([text.split() for text in data_rt['content_norm']], size=50, min_n=4, max_n=8)
w2v = gensim.models.Word2Vec([text.split() for text in data_rt['content_norm']], size=50, sg=1)

In [112]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [113]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

In [114]:
cos_w2v = cosine_distances(X_text_1_w2v,X_text_2_w2v)

In [122]:
dim = 50
data['text_1_norm'] = data['text_1'].apply(tokenize)
data['text_2_norm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

In [123]:
cos_ft = cosine_distances(X_text_1_ft,X_text_2_ft)

### Adagram

In [117]:
vm = adagram.VectorModel.load("out.pkl")

In [118]:
def get_embedding_adagram(text, model, window, dim):
    text = text.split()
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v # просто умножаем вектор на частоту
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [119]:
dim = 50
X_text_1 = np.zeros((len(data['text_1_norm']), dim))
X_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1[i] = get_embedding_adagram(text, vm, 5, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2[i] = get_embedding_adagram(text, vm, 5, dim)

In [120]:
cos_ad = cosine_distances(X_text_1,X_text_2)

In [142]:
X_text = np.concatenate([cos_svd, cos_nmf, cos_w2v, cos_ft, cos_ad], axis=1)

In [163]:
len(cos_w2v)

7227

In [144]:
y = data['label'].values

In [145]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y,random_state=1)
clf = RandomForestClassifier(n_estimators=200,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.50      0.45      0.47       629
          0       0.45      0.69      0.55       737
          1       0.39      0.11      0.17       441

avg / total       0.46      0.46      0.43      1807



In [146]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y, random_state=1)
clf = LogisticRegression(C=1, class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))


             precision    recall  f1-score   support

         -1       0.45      0.50      0.47       629
          0       0.45      0.46      0.46       737
          1       0.35      0.29      0.32       441

avg / total       0.43      0.43      0.43      1807

